In [12]:
import os
import numpy as np
import cv2
import pickle
import csv

Put all masks and images in the same folder and set folder path and csv name

In [13]:
# Folder path containing images and masks
folder_path = r'C:\Users\sebas\Documents\SAM\example_output'

# CSV file path to write the measurements
csv_filename = 'ASH.csv'
csv_file_path = os.path.join(folder_path, csv_filename)

In [14]:
def calculate_pixel_stats(image_path, mask_path):
    # Load the image in 16-bit format
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    # Load and prepare the mask
    with open(mask_path, 'rb') as f:
        mask = pickle.load(f)
        # Convert mask from boolean to 8-bit binary (0 or 255)
        mask = (mask.astype(np.uint8) * 255)

    # Apply the mask to the 16-bit image
    masked_image = cv2.bitwise_and(image, image, mask=mask)

    # Calculate the pixel statistics on the masked image
    average_intensity = np.mean(masked_image[mask == 255])  # Calculate stats only on masked area
    min_intensity = np.min(masked_image[mask == 255])
    max_intensity = np.max(masked_image[mask == 255])
    std_deviation = np.std(masked_image[mask == 255])

    return average_intensity, min_intensity, max_intensity, std_deviation

# Open the CSV file in append mode
with open(csv_file_path, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write the header row
    writer.writerow(['Filename', 'Average Intensity', 'Min Intensity', 'Max Intensity', 'Standard Deviation'])

    # Iterate over the files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            # Construct the image and mask paths
            image_path = os.path.join(folder_path, filename)
            mask_path = os.path.join(folder_path, filename[:-4] + '.pkl')

            # Calculate the pixel statistics
            average_intensity, min_intensity, max_intensity, std_deviation = calculate_pixel_stats(image_path, mask_path)

            # Write the measurements as a new row in the CSV file
            writer.writerow([filename, average_intensity, min_intensity, max_intensity, std_deviation])

    print("Pixel statistics saved to", csv_file_path)

Pixel statistics saved to C:\Users\sebas\Documents\SAM\test\ASH.csv
